# Segmenting and Clustering Neighborhoods in Toronto
### Assignement - Author : Jithin Prakash Kolamkolly
---
#### PART I
----

_Import from Wiki to Pandas_

In [1]:
#Import Pandas
import pandas as pd

#Define URL
URL='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Import to DataFrame
df_Canada =pd.read_html(URL, header=0)[0]

_1. Remove 'Not assigned' Boroughs_  
_2. Group Postal Code Neighbourhoods_  
_3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough._

In [2]:
#Option 1
df_Canada=df_Canada[df_Canada['Borough']!='Not assigned']
#Option 2
df_Canada=df_Canada.groupby("Postal Code").agg(lambda x:','.join(set(x)))
df_Canada.reset_index(inplace=True)
#Option 3
df_Canada.loc[df_Canada['Neighborhood']=="Not assigned",'Neighborhood']=df_Canada.loc[df_Canada['Neighborhood']=="Not assigned",'Borough']
#Shape of the dataframe

print("\n\nNumber of (Rows,Columns)  =  {} \n\n".format(df_Canada.shape))
df_Canada.head()



Number of (Rows,Columns)  =  (103, 3) 




,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


---
#### PART II  
----

_Importing from the CSV file provided - to the dataFrame_

In [3]:
geo_csv_path = 'https://cocl.us/Geospatial_data'
df_geo = pd.read_csv(geo_csv_path)

df_geo.head()

df_Canada = df_Canada.set_index('Postal Code').join(df_geo.set_index('Postal Code'))
df_Canada.reset_index(inplace=True)
df_Canada.head(11)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
df_Canada.shape

(103, 5)